In [1]:
import fastf1
import pandas as pd
import os

fastf1.Cache.enable_cache('../data') # Cache in /data

In [2]:
def get_race_laps(year: int, session_name: str = 'British Grand Prix') -> pd.DataFrame:
    """
    Obtiene los datos de vueltas de la carrera de Silverstone para un año dado.
    Devuelve un DataFrame con las columnas: Driver, LapTime, Sector1Time, Sector2Time, Sector3Time.

    Args:
        year (int): Año de la carrera.
        session_name (str): Nombre de la sesión (por defecto 'British Grand Prix').

    Returns:
        pd.DataFrame: DataFrame con los datos de vueltas limpios.
    """
    if not isinstance(year, int) or year < 1950:
        raise ValueError("El año debe ser un entero válido (>=1950).")
    try:
        session = fastf1.get_session(year, session_name, 'R')
        session.load()
    except Exception as e:
        raise RuntimeError(f"Error cargando la sesión: {e}")

    laps = session.laps[["Driver", "LapTime", "Sector1Time", "Sector2Time", "Sector3Time"]].copy()
    # Eliminar filas con valores nulos o tiempos no válidos
    laps = laps.dropna(subset=["Driver", "LapTime", "Sector1Time", "Sector2Time", "Sector3Time"])
    # Opcional: filtrar solo vueltas válidas (LapTime > 0)
    laps = laps[laps["LapTime"].apply(lambda x: hasattr(x, "total_seconds") and x.total_seconds() > 0)]

    return laps.reset_index(drop=True)


In [3]:
def load_session_data(year=2025, session_name='British Grand Prix'):
    session_data = get_race_laps(year, session_name)
    if session_data.empty:
        raise ValueError(f"No se encontraron datos de la carrera para {year} {session_name}.")
    return session_data

session_2024 = load_session_data(2024, 'British Grand Prix')

core           INFO 	Loading data for British Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '1', '4', '81', '55', '27', '18', '14', '23', '22', '2', '20', '3', '16', '77', '31', '11', '24', '63', '10']


In [4]:
session_2024

,Driver,LapTime,Sector1Time,Sector2Time,Sector3Time
0,HAM,0 days 00:01:31.420000,0 days 00:00:29.249000,0 days 00:00:37.314000,0 days 00:00:24.857000
1,HAM,0 days 00:01:31.716000,0 days 00:00:29.287000,0 days 00:00:37.133000,0 days 00:00:25.296000
2,HAM,0 days 00:01:31.988000,0 days 00:00:29.554000,0 days 00:00:37.258000,0 days 00:00:25.176000
3,HAM,0 days 00:01:31.677000,0 days 00:00:29.469000,0 days 00:00:37.231000,0 days 00:00:24.977000
4,HAM,0 days 00:01:31.746000,0 days 00:00:29.486000,0 days 00:00:37.195000,0 days 00:00:25.065000
...,...,...,...,...,...
936,RUS,0 days 00:01:40.405000,0 days 00:00:31.504000,0 days 00:00:40.750000,0 days 00:00:28.151000
937,RUS,0 days 00:01:41.237000,0 days 00:00:31.859000,0 days 00:00:41.018000,0 days 00:00:28.360000
938,RUS,0 days 00:01:41.902000,0 days 00:00:31.950000,0 days 00:00:41.210000,0 days 00:00:28.742000
939,RUS,0 days 00:01:43.085000,0 days 00:00:32.488000,0 days 00:00:41.387000,0 days 00:00:29.210000
